## Linear and Logistic Regression 

This notebook will try to do 2 things 

* Explain the logic and maths behind linear and logistic regression 
* Show the implementation of the two methods upon real data 

### Linear Regression 

Simple linear regression is finding a straight line which minimizes the sum of squared errors between the predicted and actual values of the dependent variable. 

For a simple data set where we have one predictor(independent variable) and one prediction(dependent variable), we would get an equation like so:

$ y = mx + c  $

This can get more complicated with multiple predictors making equations that would look like this:

$ y = ax + bz + dv + c $

#### Performing Linear Regression on real data

We will look at this data set from kaggle: 

https://www.kaggle.com/datasets/girardi69/marathon-time-predictions?resource=download

This gives lots of data about marathon runners and we will use it to predict their time

Lets take a look at the data 

In [ ]:
import pandas as pd 
import numpy as np 


pd.read_csv("MarathonData.csv")